# import, attributes and useful functions

In [9]:
from itertools import combinations 
import pandas as pd 
import numpy as np
import pattern_count
import time 
import pickle

In [10]:

def Prepatation(filename):
    mc = pd.read_csv(filename)
    mcdes = mc.describe()
    attributes = mcdes.columns.values
    return mc, mcdes, attributes


In [11]:
def DFSattributes(cur, last, comb, pattern, all_p, mcdes):
    if cur == last:
        #print("comb[{}] = {}".format(cur, comb[cur]))
        #print("{} {}".format(int(mcdes[attributes[comb[cur]]]['min']), int(mcdes[attributes[comb[cur]]]['max'])))
        for a in range(int(mcdes[attributes[comb[cur]]]['min']), int(mcdes[attributes[comb[cur]]]['max'])+1):
            s = pattern.copy()
            s[comb[cur]] = a
            all_p.append(s)
        return
    else:
        #print("comb[{}] = {}".format(cur, comb[cur]))
        #print("{} {}".format(int(mcdes[attributes[comb[cur]]]['min']), int(mcdes[attributes[comb[cur]]]['max'])))
        for a in range(int(mcdes[attributes[comb[cur]]]['min']), int(mcdes[attributes[comb[cur]]]['max'])+1):
            s = pattern.copy()
            s[comb[cur]] = a
            DFSattributes(cur+1, last, comb, s, all_p, mcdes)
        
        

def AllPatternsInComb(comb, NumAttribute, mcdes): # comb = [1,4]
    all_p = []
    pattern = [-1] * NumAttribute
    DFSattributes(0, len(comb)-1, comb, pattern, all_p, mcdes)
    return all_p


def num2string(pattern):
    st = ''
    for i in pattern:
        if i != -1:
            st += str(i)
        st += '|'
    st = st[:-1]
    return st


def P1DominatedByP2(P1, P2):
    length = len(P1)
    for i in range(length):
        if P1[i] == -1:
            if P2[i] != -1:
                return False
        if P1[i] != -1:
            if P2[i] != P1[i] and P2[i] != -1:
                return False
    return True

# coverage of P among dataset D
def cov(P, D):
    cnt = 0
    for d in D:
        if P1DominatedByP2(d, P):
            cnt += 1
    return cnt


# whether a pattern P is dominated by MUP M
def PDominatedByM(P, M):
    for m in M:
        if P1DominatedByP2(P, m):
            return True
    return False

In [12]:

def equalPattern(s, t):
    if len(s) != len(t):
        return False
    lens = len(s)
    for i in range(0, lens):
        if s[i] != t[i]:
            return False
    return True

In [16]:


time1 = time.time()


mc, mcdes, attributes = Prepatation('miss_class3.csv')
mcarray = np.array(mc)
mclist = mcarray.tolist()
NumAttribute = len(attributes)
index_list = list(range(0, NumAttribute)) # list[1, 2, ...13]


column_list_mc = np.array(mc.columns).tolist()
pc_mc = pattern_count.PatternCounter('miss_class3.csv', column_list_mc, encoded=False)
pc_mc.parse_data()

data = pd.read_csv("CleanAdult3.csv")
column_list_adult = np.array(data.columns).tolist()
pc_adult = pattern_count.PatternCounter('CleanAdult3.csv', column_list_adult, encoded=False)
pc_adult.parse_data()
dataarray = np.array(data)
datalist = dataarray.tolist()



num_pattern_checked = 0
Tha = 0.65
Thc = 50
pattern_with_low_accuracy = []
for num_att in range(1, NumAttribute+1):
    comb_num_att = list(combinations(index_list, num_att)) # list of combinations of attribute index, length num_att
    for comb in comb_num_att:
        patterns = AllPatternsInComb(comb, NumAttribute, mcdes)
        for p in patterns:
           
            num_pattern_checked += 1
            p_ = num2string(p)
            cardinality = pc_adult.pattern_count(p_)
            
            if cardinality < Thc:
                continue
            mc = pc_mc.pattern_count(p_)
            acc = 1 - mc/cardinality
   
            if acc < Tha:
                if PDominatedByM(p, pattern_with_low_accuracy) == False:
                    pattern_with_low_accuracy.append(p)
                    print("pattern_with_low_accuracy number = {}".format(len(pattern_with_low_accuracy)))

time2 = time.time()
print("execution time = %s seconds" % (time2-time1))
print(len(pattern_with_low_accuracy))
print("num_pattern_checked = ", num_pattern_checked)

pattern_with_low_accuracy number = 1
pattern_with_low_accuracy number = 2


KeyboardInterrupt: 

In [17]:
print(pattern_with_low_accuracy)

[[1, -1, 10, -1, -1, -1, 1, -1, -1, 0, -1, -1, -1], [1, -1, -1, 16, -1, -1, 1, -1, -1, 0, -1, -1, -1]]


In [19]:

with open('P30.pkl', 'rb') as filehandle:
    # read the data as binary data stream
    patterns_from_P30 = pickle.load(filehandle)
filehandle.close()


In [19]:
s = [1, -1, 10, -1, -1, -1, 1, -1, -1, 0, -1, -1, -1]

smc = (pc_mc.pattern_count(num2string(s)))
sdata = (pc_adult.pattern_count(num2string(s)))
print(sdata, smc, smc/sdata)


covmc = cov(s, mclist)
covdata = cov(s, datalist)
print(covdata, covmc, covmc/covdata)

53 22 0.41509433962264153
53 22 0.41509433962264153
